In [ ]:
# opengrid imports
from opengrid.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid import config
from opengrid.library.slack import Slack
from opengrid.library import alerts
c=config.Config()

# other imports
import pandas as pd
import json
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

In [ ]:
hp = houseprint.Houseprint()

In [ ]:
sensors = []
# Remove some sensors
listofsolar = [x.key for x in hp.search_sensors(type='electricity', system='solar')]
for key in listofsolar:
    sensors.append(hp.find_sensor(key=key))
hp.init_tmpo()
print(sensors)

In [ ]:
hp.sync_tmpos()

In [ ]:
t = []
for s in sensors:
    end = pd.Timestamp('now', tz=BXL)
    start = end - pd.Timedelta('1 day')
    df = s.get_data(head=start, tail=end, unit="kW", resample="hour").sum()
    t.append((s.key,df))
        
df_new = pd.DataFrame(t).rename(columns={0:"sensor_id", 1:"production (kWh)"}).set_index('sensor_id')

In [ ]:
df_new

# Setup NoDataBot slack bot

In [ ]:
slack_url = c.get('Slack', 'webhook')
username = 'NoSolarProdBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
title = 'No solar production'
description = 'We have not found solar production in the last 24 hours'

slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Create the alerts and send

In [ ]:
alerts.create_alerts(df_new, hp, 'no_solar_production', slack, title, description, column='production (kWh)', comparison='lower')